# Testing Interactive Notebook

I tried using ipywidgets to create an interactive plot in gyre and the package loaded, the code looked good, and everything ran---however the resulting plot wasn't interactive. I tried looking around a bit, but couldn't figure out what's going on. So I'm trying again on a new machine to see if that makes a difference.

In [19]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cmocean.cm as cmo

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, IntSlider, Dropdown

In [20]:
COUNT = xr.open_dataarray('ds_binned/COUNT.nc')

count_min = 50
mask = COUNT < count_min
masked_count = COUNT.where(mask)

In [21]:
SIG0 = xr.open_dataarray('ds_binned/SIG.nc')
levels = [26,27,27.4,27.7,27.8]

In [22]:
ct_start = 0
ct_end   = 20
ct_norm  = colors.Normalize(vmin=ct_start, vmax=ct_end)
ct_cmap  = cmo.thermal

In [23]:
sa_start = 34.5
sa_end   = 36.5
sa_norm  = colors.Normalize(vmin=sa_start, vmax=sa_end)
sa_cmap  = cmo.haline

In [24]:
sp_start = -1
sp_end   = 5
sp_norm  = colors.Normalize(vmin=sp_start, vmax=sp_end)
sp_cmap  = cmo.deep

In [25]:
ctvar_start = 10**-12
ctvar_end   = 10**-8
ctvar_norm  = colors.LogNorm(vmin=ctvar_start, vmax=ctvar_end)
var_cmap  = cmo.matter_r

In [26]:
savar_start = 10**-14
savar_end   = 10**-9
savar_norm  = colors.LogNorm(vmin=savar_start, vmax=savar_end)

In [27]:
spvar_start = 10**-13
spvar_end   = 10**-8
spvar_norm  = colors.LogNorm(vmin=spvar_start, vmax=spvar_end)

In [28]:
rat_cmap = cmo.dense

In [29]:
ds  =  COUNT

In [30]:
plot_dict = {
    'CT': {'loc': 'ds_binned/CT.nc', 'norm': ct_norm, 'cmap': ct_cmap, 'title': 'Conservative Temperature'},
    'SA': {'loc': 'ds_binned/SA.nc', 'norm': sa_norm, 'cmap': sa_cmap, 'title': 'Absolute Salinity'},
    'SP': {'loc': 'ds_binned/SP.nc', 'norm': sp_norm, 'cmap': sp_cmap, 'title': 'Spice'},
    'CT_2': {'loc': 'ds_binned/CT_2.nc', 'norm': ctvar_norm, 'cmap': var_cmap, 'title': 'Mesoscale Variance of Temperature'},
    'CT_3': {'loc': 'ds_binned/CT_3.nc', 'norm': ctvar_norm, 'cmap': var_cmap, 'title': 'Microscale Variance of Temperature'},
    'SA_2': {'loc': 'ds_binned/SA_2.nc', 'norm': savar_norm, 'cmap': var_cmap, 'title': 'Mesoscale Variance of Salinity'},
    'SA_3': {'loc': 'ds_binned/SA_3.nc', 'norm': savar_norm, 'cmap': var_cmap, 'title': 'Microscale Variance of Salinity'},
    'SP_2': {'loc': 'ds_binned/SP_2.nc', 'norm': spvar_norm, 'cmap': var_cmap, 'title': 'Mesoscale Variance of Spice'},
    'SP_3': {'loc': 'ds_binned/SP_3.nc', 'norm': spvar_norm, 'cmap': var_cmap, 'title': 'Microscale Variance of Spice'},
    'CT_2RAT': {'loc': 'ds_binned/CT_2RAT.nc', 'norm': None, 'cmap': rat_cmap, 'title': 'Ratio of Mesoscale Temperature Variance'},
    'SA_2RAT': {'loc': 'ds_binned/SA_2RAT.nc', 'norm': None, 'cmap': rat_cmap, 'title': 'Ratio of Mesoscale Salinity Variance'},
    'SP_2RAT': {'loc': 'ds_binned/SP_2RAT.nc', 'norm': None, 'cmap': rat_cmap, 'title': 'Ratio of Mesoscale Spice Variance'}}

NEED TO GET RID OF lon_center, THIS IS THE OLD ATALNTIC GRIDDING. INSTEAD NEED TO MAKE lon_center THE DIMENSION VALUE INSTEAD

In [31]:
#COUNT = xr.open_dataarray('ds_binned/COUNT.nc')

In [32]:
#COUNT = COUNT.swap_dims({'lat_center':'lat_center','lon_center':'lon_center'})

In [33]:
#COUNT

In [34]:
#COUNT.to_netcdf('ds_binned/COUNT.nc')

In [35]:
# Plotting function
def plot_full(dataset_name, depth, lat, lon):
    # Load the selected dataset
    dataset_path = plot_dict[dataset_name]['loc']
    ds = xr.open_dataarray(dataset_path)

    plt.figure(figsize=(30, 10))
    gs = plt.GridSpec(2, 2, width_ratios=[2, 1.5])
    
    ax1 = plt.subplot(gs[:, 0], projection=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.gridlines(xlocs=np.arange(-180, 180, 10), ylocs=np.arange(-90, 90, 10), draw_labels=True)
    ax1.axvline(ds.lon_center.isel(lon_center=lon).item(), color='k')
    ax1.axhline(ds.lat_center.isel(lat_center=lat).item(), color='k')
    ds.sel(PRES_INTERPOLATED=depth, method='nearest').plot(x='lon_center', y='lat_center', ax=ax1, norm=plot_dict[dataset_name]['norm'], cmap=plot_dict[dataset_name]['cmap'])
    masked_count.sel(PRES_INTERPOLATED=depth).plot.contourf(x='lon_center', y='lat_center', levels=[0, 50], hatches=['...'], colors='none', ax=ax1, add_colorbar=False)
    contour = SIG0.sel(PRES_INTERPOLATED=depth).plot.contour(x='lon_center', y='lat_center', levels=levels, colors='white', linewidths=1)
    plt.clabel(contour, fmt='%1.1f', colors='white', fontsize=10)
    ax1.set_title(f'{plot_dict[dataset_name]["title"]} at {depth}m')
    
    ax2 = plt.subplot(gs[0, 1])
    ds.isel(lon_center=lon).plot(x='lat_center', y='PRES_INTERPOLATED', ax=ax2, norm=plot_dict[dataset_name]['norm'], cmap=plot_dict[dataset_name]['cmap'])
    masked_count.isel(lon_center=lon).plot.contourf(x='lat_center', y='PRES_INTERPOLATED', levels=[0, 50], hatches=['...'], colors='none', ax=ax2, add_colorbar=False)
    contour = SIG0.isel(lon_center=lon).plot.contour(x='lat_center', y='PRES_INTERPOLATED', levels=levels, colors='white', linewidths=1)
    plt.clabel(contour, fmt='%1.1f', colors='white', fontsize=10)
    ax2.invert_yaxis()
    ax2.axhline(depth, color='k')
    ax2.set_title(f'{plot_dict[dataset_name]["title"]} at {ds.lon_center.isel(lon_center=lon).item()} lon')
    
    ax4 = plt.subplot(gs[1, 1])
    ds.isel(lat_center=lat).plot(x='lon_center', y='PRES_INTERPOLATED', ax=ax4, norm=plot_dict[dataset_name]['norm'], cmap=plot_dict[dataset_name]['cmap'])
    masked_count.isel(lat_center=lat).plot.contourf(x='lon_center', y='PRES_INTERPOLATED', levels=[0, 50], hatches=['...'], colors='none', ax=ax4, add_colorbar=False)
    contour = SIG0.isel(lat_center=lat).plot.contour(x='lon_center', y='PRES_INTERPOLATED', levels=levels, colors='white', linewidths=1)
    plt.clabel(contour, fmt='%1.1f', colors='white', fontsize=10)
    ax4.invert_yaxis()
    ax4.axhline(depth, color='k')
    ax4.set_title(f'{plot_dict[dataset_name]["title"]} at {ds.lat_center.isel(lat_center=lat).item()} lat')
    
    plt.subplots_adjust(hspace=0.55)
    plt.show()

# Create an IntSlider for depth
depth_slider = IntSlider(min=0, max=ds.PRES_INTERPOLATED.max(), step=100, value=0, description='Depth (m):')
lat_slider = IntSlider(min=0, max=len(ds.lat_center)-1, step=1, value=24, description='Lat interval:')
lon_slider = IntSlider(min=0, max=len(ds.lon_center)-1, step=1, value=18, description='Lon interval:')

# Create a Dropdown for selecting the dataset
dataset_dropdown = Dropdown(
    options=list(plot_dict.keys()),
    value='CT',
    description='Dataset:'
)

# Create interactive plot
interactive_plot_full = interactive(plot_full, dataset_name=dataset_dropdown, depth=depth_slider, lat=lat_slider, lon=lon_slider)

In [36]:
display(interactive_plot_full)

interactive(children=(Dropdown(description='Dataset:', options=('CT', 'SA', 'SP', 'CT_2', 'CT_3', 'SA_2', 'SA_…